In [1]:
# GMM, in python
import pandas as pd
import numpy as np
from scipy import optimize

# helper function
def ols(y, x):
    import numpy.linalg as la
    return la.inv(x.T @ x) @ x.T @ y


In [3]:
# load the data
rev = pd.read_csv("/Users/JuliaYI/Desktop/TA/homework_2022/hw3_data/revfunc_data.csv")
price = pd.read_csv("/Users/JuliaYI/Desktop/TA/homework_2022/hw3_data/price_data.csv")
dt = pd.merge(rev, price, how='inner', on=('year'))
dt.year = pd.to_datetime(dt.year, format="%Y")
dt = dt.set_index('year')
dt = dt.to_period("Y")
dt['lagprice'] = dt.groupby('firm_id')['price'].shift(1)
dt['lag1pi'] = dt.groupby('firm_id')['output'].shift(1)
dt['lag1k'] = dt.groupby('firm_id')['capital'].shift(1)
#dt['lag2pi'] = dt.groupby('firm_id')['output'].shift(2)
#dt['lag2k'] = dt.groupby('firm_id')['capital'].shift(2)

dt = dt.sort_values(['firm_id','year'])
dt = dt.dropna()

# quasi-first-differenicng
X = np.log(dt.capital.to_numpy()).reshape(145000,1) #k
X = np.hstack((X, np.log(dt.lag1pi.to_numpy()).reshape(145000,1))) #lagpi
X = np.hstack((X, np.log(dt.lag1k.to_numpy()).reshape(145000,1))) #lagk
X = np.hstack((X, np.ones((145000,1)))) #constant
X = np.hstack((X, np.log(dt.output.to_numpy()).reshape(145000,1))) #pi
est = ols(X[:,-1], X[:,:-1])
print(est)


[ 0.64889678  0.74368167 -0.45614643  0.17717113]


In [4]:
# GMM
def MM(α, ρ, data):
    mm = np.empty(2)
    qderror = np.log(data.output) - ρ * np.log(data.lag1pi) - \
        α * (np.log(data.capital) - ρ * np.log(data.lag1k))
    loga = np.log(data.output) - α * np.log(data.capital)

    mm[0] = np.mean(qderror * data.lag1k)
    mm[1] = np.mean(qderror * data.lag1pi)
    #mm[2] = np.mean(loga * data.price)
    #mm[3] = np.mean(loga * data.lagprice)

    return mm

def Jfunc(params, W, data):
    return MM(*params, data) @ W @ MM(*params, data)


In [5]:
# first round: use identity matrix as weighting matrix
W_mat = np.eye(2)

x0 = (0.5, 0.8)
bnds = ((0.5, 0.99), (0.6, 0.99))
tol = 1e-5
opt={'maxiter': 10_00, 'disp': False}
params = optimize.minimize(Jfunc, x0, args=(W_mat, dt), method='Powell',
                               bounds=bnds, options=opt, tol=tol)

print(f"The estimate of α and ρ are {params.x[0]:.4f} and {params.x[1]:.4f}, respectively.")

price = dt.price.to_numpy()
lagprice = dt.lagprice.to_numpy()

# second round: use a better weighting matrix
def boot(N_sample, T_sample, data, α, ρ):
    """
    obtain VCV of sample moments by bootstrap.

    * N_sample: number of sample
    * T_sample: length of sample, should be the same with the data period, here is 5000 firms×(30-1) periods
    * data: observed data, columns:(k, lagpi, lagk, constant, pi)
    * α and ρ: what we got from the previous step
    """
    total = N_sample * T_sample #total number of draws
    sample_idx = np.random.default_rng().random(total) * T_sample
    sample_idx = sample_idx.astype(int)
    sample_idx = sample_idx.reshape((N_sample, T_sample))
    mm_sample = np.empty((N_sample, 2)) #column is different moment

    for n in range(N_sample):
        sample = np.take(data, sample_idx[n,:], axis=0)
        qderror = sample[:,4] - ρ * sample[:,1] - α * (sample[:,0] - ρ * sample[:,2])
        loga = sample[:,4] - α * sample[:,0] #logpi - αlogk
        mm0 = np.mean(qderror * sample[:,2])
        mm1 = np.mean(qderror * sample[:,1])
        #mm2 = np.mean(loga * price)
        #mm3 = np.mean(loga * lagprice)

        mm_sample[n,:] = np.array([mm0, mm1])
    return np.cov(mm_sample, rowvar=False)

α1, ρ1 = params.x
Ω = boot(500, 145000, X, α1, ρ1)
W1 = np.linalg.inv(Ω)
W1 = W1 / np.sum(W1)

params_new = optimize.minimize(Jfunc, x0, args=(W1, dt), method='Powell',
                               bounds=bnds, options=opt, tol=tol)

print(f"The estimate of α and ρ are {params_new.x[0]:.4f} and {params_new.x[1]:.4f}, respectively.")


The estimate of α and ρ are 0.6023 and 0.8060, respectively.
The estimate of α and ρ are 0.6018 and 0.7959, respectively.


In [8]:
# recursive method
def GMM(α0, ρ0, N_boot=100):
    "wrap up the two-step procedures"
    Ω1 = boot(N_boot, 145000, X, α0, ρ0)
    W1 = np.linalg.inv(Ω1)
    W1 = W1 / np.sum(W1)
    α1, ρ1 = optimize.minimize(Jfunc, x0, args=(W1, dt), method='Powell',
                                   bounds=bnds, options=opt, tol=tol).x

    return α1, ρ1

def RecursiveGMM(α0, ρ0, tol=1e-4, max_iter=1000, show_it=True):
    error = 1
    it = 0
    α, ρ = α0, ρ0
    while error > tol and it < max_iter:
        α1, ρ1 = GMM(α, ρ)
        error = np.amax(np.abs(np.array([α1, ρ1])- np.array([α, ρ])))
        α, ρ = α1, ρ1
        it += 1
        if show_it == True:
            print(f"VF iter = {it}; VF error = {error}.")
            print(f"α = {α1}, ρ = {ρ1}")
    if it == max_iter:
        print(f"Fail to converge within {max_iter} iterations.")
    elif it < max_iter:
        print(f"Converged in {it} iterations.")

    return α1, ρ1

RecursiveGMM(0.6018, 0.7959)

VF iter = 1; VF error = 0.00028599667286555164.
α = 0.6018380144391031, ρ = 0.7956140033271345
VF iter = 2; VF error = 0.0001436004490018572.
α = 0.6018448075198572, ρ = 0.7957576037761364
VF iter = 3; VF error = 4.938630324735627e-05.
α = 0.6018471084582764, ρ = 0.7958069900793837
Converged in 3 iterations.


(0.6018471084582764, 0.7958069900793837)